In [ ]:
import unicodedata
import neologdn
import emoji
import re
import pandas as pd

In [ ]:
def remove_url(text:str) -> str:
    t = re.sub(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+\$,%#]+)", " ", text)
    return t

def normalization_string(text:str) -> str: 
	text = unicodedata.normalize("NFKC", text) # UNICODE正規化
	text = neologdn.normalize(text)            # NEOLOGD正規化
	text = remove_url(text)                    # URL削除
	# text = demoji.replace(text, ' ')           # 絵文字削除
	text = emoji.demojize(text)                # 絵文字をテキストに変換
	text = text.lower()                        # 小文字化
    
	return text

In [ ]:
pdf_features = pd.read_parquet("tmp/topic_features_navit.parquet")
pdf_features = pdf_features.sort_values(["topic_id", "feature_id"], ascending=True, ignore_index=True)
pdf_features["type_caption"]  = pdf_features["type_caption"].apply(normalization_string)

pdf_master = pd.read_csv("tmp/nivit_master.csv")
pdf_master["BUSINESS_NAME_S"] = pdf_master["BUSINESS_NAME_S"].apply(normalization_string)

pdf_navit = pd.merge(pdf_master, pdf_features, how='inner', left_on='BUSINESS_NAME_S', right_on='type_caption')
pdf_navit = pdf_navit[["pattern", "topic_id", "feature_id", "BUSINESS_CODE", "BUSINESS_NAME_L", "BUSINESS_NAME_M", "BUSINESS_NAME_S", "value"]]
pdf_navit = pdf_navit.astype({"pattern":str, "topic_id":int, "feature_id":int, "BUSINESS_CODE":int, "BUSINESS_NAME_L":str, "BUSINESS_NAME_M":str, "BUSINESS_NAME_S":str, "value":float})
pdf_navit = pdf_navit.sort_values(["pattern", "topic_id", "feature_id", "BUSINESS_CODE", "BUSINESS_NAME_L", "BUSINESS_NAME_M", "BUSINESS_NAME_S"], ascending=True, ignore_index=True)

pdf_navit.to_csv("tmp/cohort_category.csv", header=True, index=False)

pdf_navit

In [ ]:
pdf_navit = pd.read_parquet("tmp/topic_features_navit.parquet")
pdf_navit